In [1]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Label
from bokeh.models.widgets import Slider, TextInput
from bokeh.layouts import row, widgetbox
from bokeh.io import push_notebook
from scipy.stats import beta
from scipy.integrate import quad
from scipy.special import gammaln
import numpy as np

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def plot_pdfs(x, prior, posterior, title=""):
    source = ColumnDataSource({'x': x, 'prior': prior.pdf(x), 'posterior': posterior.pdf(x)})
    
    p = figure(height=400, width=500, toolbar_location=None)
    p.line(source=source, x='x', y='prior', line_width=4, line_alpha=0.6, legend='Prior')
    p.line(source=source, x='x', y='posterior', line_width=3, line_alpha=0.6, legend='Posterior', color='firebrick')

    p.title.text = title
    p.title.text_font_size='14pt'
    p.xaxis.axis_label = 'q'
    p.yaxis.axis_label = 'Probability Density'
#     p.axis.axis_label_text_font_style = 'normal'
    p.axis.axis_label_text_font_size = '16pt'
    p.axis.major_label_text_font_size = '14pt'
    p.outline_line_width = 1
    p.outline_line_alpha = 1
    p.outline_line_color = "black"

    return p

## Parameter Estimation
Given a set of coin flips, what is the coin's weighting in favor of Heads?

In [46]:
x = np.linspace(0, 1, 500)  # x-axis data

# Data: coin flip outcomes
n_heads = 115  # Data: number of heads observed
n_tails = 85  # Data: number of tails observed

# Completely uninformative prior
a = 1  # Prior hyperparameter: pseudo-counts for number of heads (>= 1)
b = 1  # Prior hyperparameter: pseudo-counts of number of tails (>= 1)

# PDFs for prior and posterior
prior_uninformative = beta(a, b)
posterior_uninformative = beta(a + n_heads, b + n_tails)
p1 = plot_pdfs(x, prior_uninformative, posterior_uninformative, title='Uninformative Prior')

# Weak prior in favor of fair coin
a = 5  # Prior hyperparameter: pseudo-counts for number of heads (>= 1)
b = 5  # Prior hyperparameter: pseudo-counts of number of tails (>= 1)

# PDFs for prior and posterior
prior_weak = beta(a, b)
posterior_weak = beta(a + n_heads, b + n_tails)
p2 = plot_pdfs(x, prior_weak, posterior_weak, title='Weak prior in favor of fair coin')

# Strong prior in favor of fair coin
a = 1000  # Prior hyperparameter: pseudo-counts for number of heads (>= 1)
b = 1000  # Prior hyperparameter: pseudo-counts of number of tails (>= 1)

# PDFs for prior and posterior
prior_strong = beta(a, b)
posterior_strong = beta(a + n_heads, b + n_tails)
p3 = plot_pdfs(x, prior_strong, posterior_strong, title='Strong prior in favor of fair coin')

# Set up layout
layout = row(p1, p2, p3)
show(layout)

In [43]:
# Approximate the probability that Prob(Heads) lies between 0.45 and 0.55
lower_bound = 0.45
upper_bound = 0.55
print(quad(prior_weak.pdf, lower_bound, upper_bound))
print(quad(posterior_weak.pdf, lower_bound, upper_bound))

(0.24284189089843758, 2.6960865861895563e-15)
(0.2637839437174284, 2.9285900784151545e-15)


In [42]:
posterior_weak.mean(), posterior_weak.std()

(0.5714285714285714, 0.03406837000787847)

## Model Selection 1: Is the coin fair?

Suppose the following:
* We have a set of coin flip data $D = (h, t)$, where $h$ is the number of heads and $t$ is the number of tails.
* The unknown parameter $q$ is the probability that any given coin flip outcome will be heads
* Let $H_0$ represent the hypothesis "The coin is fair; $q = 0.5$"
* Let $H_1$ represent the hypothesis "The coin is unfair; $q$ is some value between 0 and 1"

In [7]:
# P(q=0.5) = gamma(H+T+1)/(gamma(H+1) * gamma(T+1)) * 0.5^H * 0.5^T
# P(q=x) = gamma(H+T+1)/(gamma(H+1) * gamma(T+1)) * gamma(a+b)/(gamma(a) * gamma(b)) * gamma(H+a) * gamma(T+b) / gamma(H+a+T+b)

In [9]:
# Data
n_heads = 115
n_tails = 85

# Prior probabilities for each model
prior_fair = 0.5
prior_unfair = 1 - prior_fair

# Prior hyperparameters on the unfair coin model
a = 1
b = 1

# Bias weighting for heads for the fair coin model
q = 0.5

binom_coeff = gammaln(n_heads + n_tails + 1) - gammaln(n_heads + 1) - gammaln(n_tails + 1)
p_fair = binom_coeff + (n_heads * np.log(q)) + (n_tails * np.log((1 - q))) + np.log(prior_fair)
p_unfair = binom_coeff + gammaln(a + b) - gammaln(a) - gammaln(b) + gammaln(n_heads + a) + gammaln(n_tails + b) - gammaln(n_heads + n_tails + a + b) + np.log(prior_unfair)

log_bayes_factor = p_unfair - p_fair  # If > 0, favor unfair coin; if < 0, favor fair coin
print(log_bayes_factor)   # Weakly favors fair coin

-0.17993064270033443


In [10]:
p = figure(height=80, width=700, toolbar_location=None)
# p.outline_line_alpha = 0

label_text = f'log(Bayes Factor): {log_bayes_factor:0.2f}'
bf_fair_vs_unfair = Label(x=0, y=15, text=label_text, x_units='screen', y_units='screen', render_mode='css', text_font_size='22pt', text_font_style='bold')
p.add_layout(bf_fair_vs_unfair)
show(p)

### Compare to classical maximum likelihood ratio

In [11]:
q_fair = 0.5
log_likelihood_fair = binom_coeff + (n_heads * np.log(q_fair)) + (n_tails * np.log((1 - q_fair)))

q_unfair = n_heads / (n_heads + n_tails)
log_likelihood_unfair = binom_coeff + (n_heads * np.log(q_unfair)) + (n_tails * np.log((1 - q_unfair)))

log_likelihood_ratio = log_likelihood_unfair - log_likelihood_fair
print(log_likelihood_ratio)  # If > 0, favor unfair coin; if < 0, favor fair coin

2.25851436583239


### Compare to classical hypothesis testing

In [12]:
# Defining the unfair coin scenario as the alternative hypothesis, the p-value is 0.02 < 0.05, so test favors an unfair coin.

## Model Selection 2: Is this the same coin?

* Suppose we have two sets of coin flip data, $D_1 = (h_1, t_1)$ and $D_2 = (h_2, t_2)$, and want to determine if they came from the same coin
* Let $H_0$ represent the hypothesis "$D_1$ and $D_2$ come from the same coin; $q_1 = q_2$"
* Let $H_1$ represent the hypothesis "$D_1$ and $D_2$ come from different coins; $q_1 \neq q_2$"

In [38]:
# Data
h1 = 115
t1 = 85
h2 = 215
t2 = 150

# Prior probabilities for each model
prior_same = 0.5
prior_different = 1 - prior_fair

a = 3
b = 3

p_same = gammaln(a + b) - gammaln(a) - gammaln(b) + \
         gammaln(h1 + h2 + a) + gammaln(t1 + t2 + b) - gammaln(h1 + h2 + t1 + t2 + a + b) + \
         np.log(prior_same)
         
p_different = gammaln(a + b) - gammaln(a) - gammaln(b) + \
              gammaln(h1 + a) + gammaln(t1 + b) - gammaln(h1 + t1 + a + b) + \
              gammaln(a + b) - gammaln(a) - gammaln(b) + \
              gammaln(h2 + a) + gammaln(t2 + b) - gammaln(h2 + t2 + a + b) + \
              np.log(prior_different)

log_bayes_factor = p_different - p_same
print(log_bayes_factor)  # If > 0, favor different coins; if < 0, favor same coin

-1.604029456522312


In [45]:
# Compare to chi-square test?